In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

In [2]:
load_dotenv(find_dotenv('../application/.env'))

True

In [ ]:
endpoint = os.environ["SEARCH_ENDPOINT"]
key = os.environ["SEARCH_API_KEY"]

print(key)

client = SearchIndexClient(endpoint, AzureKeyCredential(key))

In [ ]:
name = "restaurant"
fields = [
    SimpleField(name="restaurantId", type=SearchFieldDataType.String, key=True),
    SimpleField(name="averageCost", type=SearchFieldDataType.Double),
    SearchableField(name="description", type=SearchFieldDataType.String),
    ComplexField(name="address", fields=[
        SimpleField(name="streetAddress", type=SearchFieldDataType.String),
        SimpleField(name="city", type=SearchFieldDataType.String),
    ])
]

In [ ]:
from azure.core.exceptions import HttpResponseError

cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []

index = SearchIndex(
    name=name,
    fields=fields,
    scoring_profiles=scoring_profiles,
    cors_options=cors_options)


try:
    result = client.create_index(index)
    print(f"Index '{name}' created.")
except HttpResponseError as e:
    print(f"Index '{name}' already exists.")


### Add documents to the index

In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

index_name = "restaurant"
endpoint = os.environ["SEARCH_ENDPOINT"]
key = os.environ["SEARCH_API_KEY"]

documents = [
    {
        'restaurantId': '1',
        'averageCost': 50.0,
        'description': 'Traditional Italian cuisine with a modern twist.',
        'address': {
            'streetAddress': '123 Via Roma',
            'city': 'Rome',
        }
    },
    {
        'restaurantId': '2',
        'averageCost': 70.0,
        'description': 'Family-friendly Italian restaurant with classic dishes.',
        'address': {
            'streetAddress': '456 Via Milano',
            'city': 'Milan',
        }
    },
    {
        'restaurantId': '3',
        'averageCost': 35.0,
        'description': 'Cozy trattoria offering regional specialties.',
        'address': {
            'streetAddress': '789 Via Napoli',
            'city': 'Naples',
        }
    }
]

search_client = SearchClient(endpoint, index_name, AzureKeyCredential(key))
result = search_client.upload_documents(documents=documents)


In [ ]:
for res in result:
    print(f"Upload of document with ID '{res.key}' succeeded: {res.succeeded}")

Now we can retrieve Documents from ACS

In [ ]:
results = client.search(search_text="Family friendly?")

for result in results:
    print(result)

### Using LangChain with ACS

In [3]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

In [5]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)
index_name: str = "langchain-example"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.environ.get("SEARCH_ENDPOINT"),
    azure_search_key=os.environ.get("SEARCH_API_KEY"),
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [7]:
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader('./restaurant', glob="**/*.txt", loader_cls=TextLoader)
data = loader.load()
print(len(data))

2


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 120,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)
docs = text_splitter.split_documents(data)
print(len(docs))

39


In [9]:
vector_store.add_documents(documents=docs)

['ODdjODkxMDgtMTg5Yy00MWY3LWEyMTktZDIxMjA4YjlkZDNk',
 'NzBkZmVkODEtN2M4My00Y2VkLTkxZmMtOGVjNGI4M2ViMmI5',
 'ZmE3NzQ3YWUtYzMwYi00YzFhLTgwMzctMzRhOTQwZDY0MGY2',
 'ZmU4ZjQ3ZTYtYmJiMy00YjhmLWE3ZjAtNThlMTE0MjNiZTE3',
 'YWVlOWZmOGQtMTYyZi00YzlkLWI4NjUtODYzMDM5ZjdhY2Y3',
 'Y2IxYWYwYzUtN2VmZi00MTM0LTlkODgtOTgxOTYyZjhhNDUz',
 'MWYwOGE0YTQtMTgwZi00MTNjLTkxODUtNmIzNDYxY2YxM2Zj',
 'NmVmNTRkMjUtMTNmNC00ZGJkLWFjOWItYjE4ODkxZWY3ZGYy',
 'NDU3MDRhY2UtNDBhNS00YThhLWFhOGItNzFlN2MwZDQwNmE0',
 'OTVhNTI1ZTEtYTE4ZC00YTdhLWIyZTctYjVhNjBiYWYzYTMy',
 'ZTNkNTQwN2YtZWZlOS00MWM2LWE1ODktYmY0ZTEwOTBmOTJi',
 'MzE1NTAzODQtYjI0YS00ZTA2LWJlNDMtMjI0YmYxNDlhY2Jh',
 'ODczOGQ0YjMtMTJhOS00Nzc0LTk0MDMtNjZlYWY3YWEzZWU4',
 'NzdjYzk4YzAtMTE1Ny00MzMxLTkyMTYtNjlkMDI2Yzc4ODgz',
 'YWRlMDhmYWMtM2ZmMS00NmZlLThiYzctZjI3MWY1YzA3OTdl',
 'ZjVhNjQ2N2MtMTM2Yy00NGJkLTg2NDQtZTRkOWYyYThlM2M0',
 'YTQyMTQ2ZjktZTQxNC00YmZjLTk5ZjYtNDEzNWQ1NGNkYzU0',
 'NDJkMWU2ZjYtMTZhZC00OWExLWJlNjMtN2M0YTdhZTI0YzQ0',
 'YmM0NWMwMWItZmFlOS00NjM1LThjOGYtNzY0OTgzNDc1

In [17]:

docs = vector_store.similarity_search(
    query="When are the opening hours of the restaurant?",
    k=3,
    search_type="similarity",
)
print(docs)


[Document(page_content='Restaurant Opening Hours:', metadata={'id': 'YmFmZWY5YTItZWNkMi00NjdjLThmZWItYjQ3YTU2OTRjMmEw', 'source': 'restaurant\\opening_hours.txt'}), Document(page_content='Special Hours: Our kitchen closes 30 minutes before the restaurant closing time.', metadata={'id': 'MDRhYmNjZjAtYmZlYS00M2Q3LTlkYTAtNDlkOWQzOWUwNTM4', 'source': 'restaurant\\opening_hours.txt'}), Document(page_content='Monday to Thursday: 11:00 AM - 11:00 PM\nFriday: 11:00 AM - 12:00 AM (midnight)', metadata={'id': 'ZTc3NGVhNTMtNzUxMi00MjkzLTg0NTktMTFjN2Q2YTIwNGVm', 'source': 'restaurant\\opening_hours.txt'})]


In [19]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), chain_type="stuff", retriever=vector_store.as_retriever())
qa.run("When are the opening hours of the restaurant?")

'The opening hours of the restaurant are Monday to Thursday from 11:00 AM to 11:00 PM, and on Friday from 11:00 AM to 12:00 AM (midnight).'